# 🔀 Hybrid Retrieval Demo: Vector DB + Knowledge Graph

This notebook demonstrates **hybrid retrieval** combining:
- **Vector Database (ChromaDB)**: Semantic similarity search using embeddings
- **In-Memory Knowledge Graph (NetworkX)**: Graph-based traversal and relationship reasoning

## Why Hybrid Retrieval?

| Approach | Strengths | Weaknesses |
|----------|-----------|------------|
| **Vector Search** | Semantic similarity, handles paraphrasing | Misses structural relationships |
| **Graph Search** | Captures relationships, multi-hop reasoning | Limited semantic understanding |
| **Hybrid** | Best of both worlds | More complex implementation |

## Architecture Overview

```
Query → [Vector Search] → Semantic Matches
     ↘                   ↗
       [Fusion Strategy] → Final Results
     ↗                   ↘
Query → [Graph Search]  → Relational Matches
```

---


## 📦 1. Installation

Install the required libraries for vector database, graph processing, and embeddings.


In [ ]:
# Install dependencies
%pip install chromadb sentence-transformers networkx matplotlib --quiet
%pip install numpy pandas scikit-learn --quiet
%pip install openai tiktoken --quiet

print("✅ Dependencies installed!")


In [ ]:
import os
import json
import re
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Any, Set
from dataclasses import dataclass, field
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

print("✅ Libraries imported!")


## 🔧 2. Configuration & Sample Data

We'll use sample data about a fictional tech company (same as the original GraphRAG demo) to demonstrate hybrid retrieval.


In [ ]:
# Sample documents about TechCorp (enriched version)
DOCUMENTS = [
    {
        "id": "doc_1",
        "title": "Company Overview",
        "content": """TechCorp is a leading technology company founded in 2015 by Sarah Chen and Michael Rodriguez 
        in San Francisco. The company specializes in artificial intelligence solutions for enterprise customers. 
        With over 5,000 employees across 20 offices worldwide, TechCorp has become a major player in the AI industry.
        Sarah Chen serves as the CEO and has led the company through multiple successful funding rounds."""
    },
    {
        "id": "doc_2", 
        "title": "Leadership Team",
        "content": """Sarah Chen serves as the CEO and has led the company through multiple successful funding rounds.
        She previously worked at Google and Stanford AI Lab. Michael Rodriguez, the CTO, oversees all technical 
        operations and R&D. He holds a PhD in Machine Learning from MIT. The CFO, Jennifer Park, joined in 2019 
        from Goldman Sachs. She has been instrumental in the company's financial growth and successful IPO in 2023.
        David Thompson leads the Sales division and has expanded the customer base to include Fortune 500 companies."""
    },
    {
        "id": "doc_3",
        "title": "Products and Services", 
        "content": """TechCorp's flagship product, AIAssist Pro, is an enterprise AI assistant that helps companies 
        automate customer service operations. It uses advanced natural language processing and has been deployed 
        by over 200 enterprise customers. DataSense Analytics is the company's second major product, offering 
        predictive analytics for supply chain optimization. Major clients include Walmart and Target, who have 
        reported 30% efficiency improvements. The newest product, SecureAI, launched in 2024, focuses on 
        AI-powered cybersecurity. It has attracted partnerships with JPMorgan Chase, Bank of America, and Wells Fargo."""
    },
    {
        "id": "doc_4",
        "title": "Research and Development",
        "content": """TechCorp's R&D division, led by Dr. Emily Watson, has published over 50 papers in top AI 
        conferences. The team recently made a breakthrough in efficient transformer architectures, reducing 
        compute costs by 40%. The company collaborates with Stanford University, MIT, and Carnegie Mellon on 
        various research projects. Dr. Watson's team includes researchers from DeepMind, OpenAI, and Google Brain."""
    },
    {
        "id": "doc_5",
        "title": "Financial Performance",
        "content": """In 2024, TechCorp reported revenue of $2.5 billion, a 45% increase from the previous year.
        The company's market cap reached $50 billion after the successful IPO. Major investors include 
        Sequoia Capital, Andreessen Horowitz, and SoftBank Vision Fund. The IPO was led by Jennifer Park 
        and raised significant capital for expansion."""
    },
    {
        "id": "doc_6",
        "title": "Future Plans",
        "content": """TechCorp plans to expand into the healthcare AI market in 2025, with partnerships already 
        in place with Mayo Clinic and Cleveland Clinic. The company is also developing autonomous systems for 
        logistics, working with FedEx and UPS on pilot programs. Sarah Chen announced plans to open new R&D 
        centers in London, Singapore, and Tel Aviv to attract global talent and serve international customers."""
    },
    {
        "id": "doc_7",
        "title": "Technology Stack",
        "content": """TechCorp's AI platform is built on a proprietary transformer architecture developed by 
        Michael Rodriguez and his team. The platform runs on cloud infrastructure from AWS and Google Cloud.
        AIAssist Pro uses a fine-tuned large language model with over 100 billion parameters. The system 
        processes over 10 million customer queries daily across all deployed instances."""
    },
    {
        "id": "doc_8",
        "title": "Partnerships and Clients",
        "content": """TechCorp has strategic partnerships with major tech companies including Amazon, Microsoft, 
        and Google. The company serves over 500 enterprise clients across various industries. Key clients 
        include Walmart, Target, JPMorgan Chase, Bank of America, Wells Fargo, FedEx, and UPS. The sales 
        team led by David Thompson has grown revenue from enterprise clients by 60% year-over-year."""
    }
]

print(f"📚 Loaded {len(DOCUMENTS)} documents")
for doc in DOCUMENTS:
    print(f"   - {doc['title']}: {len(doc['content'])} chars")


## 🏗️ 3. Core Data Structures

Define the data classes for entities, relationships, and retrieval results.


In [ ]:
@dataclass
class Entity:
    """Represents an extracted entity from the knowledge base."""
    id: str
    name: str
    type: str  # PERSON, ORGANIZATION, PRODUCT, LOCATION, etc.
    description: str
    source_docs: List[str] = field(default_factory=list)
    attributes: Dict[str, Any] = field(default_factory=dict)
    
    def to_text(self) -> str:
        """Convert entity to searchable text."""
        return f"{self.name} ({self.type}): {self.description}"


@dataclass 
class Relationship:
    """Represents a relationship between two entities."""
    id: str
    source: str  # Entity ID
    target: str  # Entity ID
    relation_type: str  # WORKS_AT, FOUNDED, DEVELOPS, etc.
    description: str
    weight: float = 1.0
    source_docs: List[str] = field(default_factory=list)
    
    def to_text(self) -> str:
        """Convert relationship to searchable text."""
        return f"{self.source} {self.relation_type} {self.target}: {self.description}"


@dataclass
class RetrievalResult:
    """Represents a single retrieval result from any source."""
    id: str
    content: str
    score: float
    source: str  # 'vector', 'graph', or 'hybrid'
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def __repr__(self):
        return f"RetrievalResult(id={self.id}, score={self.score:.3f}, source={self.source})"


@dataclass
class HybridResult:
    """Aggregated result from hybrid retrieval."""
    query: str
    vector_results: List[RetrievalResult]
    graph_results: List[RetrievalResult]
    fused_results: List[RetrievalResult]
    fusion_method: str
    
    def summary(self) -> str:
        return (f"Query: {self.query}\n"
                f"Vector results: {len(self.vector_results)}\n"
                f"Graph results: {len(self.graph_results)}\n"
                f"Fused results: {len(self.fused_results)}\n"
                f"Fusion method: {self.fusion_method}")


print("✅ Data classes defined!")


## 🗄️ 4. Vector Store (ChromaDB)

The Vector Store handles semantic similarity search using embeddings. We use ChromaDB as an in-memory vector database with sentence transformers for embeddings.


In [ ]:
class VectorStore:
    """
    Vector database for semantic similarity search using ChromaDB.
    
    Supports multiple collections:
    - documents: Full document chunks
    - entities: Entity descriptions
    - relationships: Relationship descriptions
    """
    
    def __init__(self, embedding_model: str = "all-MiniLM-L6-v2"):
        """Initialize the vector store with an embedding model."""
        self.embedding_model_name = embedding_model
        self.embedding_model = SentenceTransformer(embedding_model)
        self.embedding_dim = self.embedding_model.get_sentence_embedding_dimension()
        
        # Initialize ChromaDB client (in-memory)
        self.client = chromadb.Client()
        self.collections: Dict[str, chromadb.Collection] = {}
        
        print(f"✅ VectorStore initialized with '{embedding_model}'")
        print(f"   Embedding dimension: {self.embedding_dim}")
    
    def create_collection(self, name: str) -> chromadb.Collection:
        """Create or get a collection."""
        if name in self.collections:
            return self.collections[name]
        
        collection = self.client.get_or_create_collection(
            name=name,
            metadata={"hnsw:space": "cosine"}  # Use cosine similarity
        )
        self.collections[name] = collection
        return collection
    
    def add_documents(self, collection_name: str, documents: List[Dict[str, Any]]):
        """
        Add documents to a collection.
        
        Args:
            collection_name: Name of the collection
            documents: List of dicts with 'id', 'content', and optional 'metadata'
        """
        collection = self.create_collection(collection_name)
        
        ids = [doc['id'] for doc in documents]
        texts = [doc['content'] for doc in documents]
        metadatas = [doc.get('metadata', {}) for doc in documents]
        
        # Generate embeddings
        embeddings = self.embedding_model.encode(texts, show_progress_bar=False).tolist()
        
        # Add to collection
        collection.add(
            ids=ids,
            embeddings=embeddings,
            documents=texts,
            metadatas=metadatas
        )
        
        print(f"   Added {len(documents)} items to '{collection_name}'")
    
    def add_entities(self, entities: List[Entity]):
        """Add entities to the entities collection."""
        docs = [{
            'id': e.id,
            'content': e.to_text(),
            'metadata': {
                'name': e.name,
                'type': e.type,
                'source_docs': ','.join(e.source_docs)
            }
        } for e in entities]
        self.add_documents('entities', docs)
    
    def add_relationships(self, relationships: List[Relationship]):
        """Add relationships to the relationships collection."""
        docs = [{
            'id': r.id,
            'content': r.to_text(),
            'metadata': {
                'source': r.source,
                'target': r.target,
                'relation_type': r.relation_type
            }
        } for r in relationships]
        self.add_documents('relationships', docs)
    
    def search(self, collection_name: str, query: str, k: int = 5) -> List[RetrievalResult]:
        """
        Search a collection for similar documents.
        
        Args:
            collection_name: Name of the collection to search
            query: Query text
            k: Number of results to return
            
        Returns:
            List of RetrievalResult objects
        """
        if collection_name not in self.collections:
            raise ValueError(f"Collection '{collection_name}' not found")
        
        collection = self.collections[collection_name]
        
        # Generate query embedding
        query_embedding = self.embedding_model.encode([query], show_progress_bar=False).tolist()
        
        # Search
        results = collection.query(
            query_embeddings=query_embedding,
            n_results=min(k, collection.count()),
            include=['documents', 'metadatas', 'distances']
        )
        
        # Convert to RetrievalResult objects
        retrieval_results = []
        for i in range(len(results['ids'][0])):
            # Convert distance to similarity score (ChromaDB returns L2 or cosine distance)
            distance = results['distances'][0][i]
            # For cosine distance in ChromaDB, similarity = 1 - distance
            similarity = 1 - distance
            
            retrieval_results.append(RetrievalResult(
                id=results['ids'][0][i],
                content=results['documents'][0][i],
                score=similarity,
                source='vector',
                metadata=results['metadatas'][0][i] if results['metadatas'] else {}
            ))
        
        return retrieval_results
    
    def get_embedding(self, text: str) -> np.ndarray:
        """Get the embedding for a text."""
        return self.embedding_model.encode([text], show_progress_bar=False)[0]


print("✅ VectorStore class defined!")


## 🕸️ 5. Knowledge Graph (NetworkX)

The Knowledge Graph handles relationship-based retrieval using graph traversal algorithms. We use NetworkX for in-memory graph operations.


In [ ]:
class KnowledgeGraph:
    """
    In-memory knowledge graph using NetworkX for graph-based retrieval.
    
    Supports:
    - Entity and relationship storage
    - Multi-hop graph traversal
    - Subgraph extraction
    - Community detection
    """
    
    def __init__(self):
        """Initialize an empty knowledge graph."""
        self.graph = nx.DiGraph()  # Directed graph for relationships
        self.entities: Dict[str, Entity] = {}
        self.relationships: Dict[str, Relationship] = {}
        self.entity_name_to_id: Dict[str, str] = {}
        
        print("✅ KnowledgeGraph initialized")
    
    def add_entity(self, entity: Entity):
        """Add an entity to the graph."""
        self.entities[entity.id] = entity
        self.entity_name_to_id[entity.name.lower()] = entity.id
        
        # Add node with attributes
        self.graph.add_node(
            entity.id,
            name=entity.name,
            type=entity.type,
            description=entity.description
        )
    
    def add_relationship(self, relationship: Relationship):
        """Add a relationship to the graph."""
        self.relationships[relationship.id] = relationship
        
        # Add edge with attributes
        self.graph.add_edge(
            relationship.source,
            relationship.target,
            id=relationship.id,
            relation_type=relationship.relation_type,
            description=relationship.description,
            weight=relationship.weight
        )
    
    def get_entity_by_name(self, name: str) -> Optional[Entity]:
        """Get an entity by its name (case-insensitive)."""
        entity_id = self.entity_name_to_id.get(name.lower())
        return self.entities.get(entity_id) if entity_id else None
    
    def get_neighbors(self, entity_id: str, depth: int = 1, 
                      direction: str = 'both') -> Set[str]:
        """
        Get neighboring entities up to a certain depth.
        
        Args:
            entity_id: Starting entity ID
            depth: Maximum traversal depth
            direction: 'in', 'out', or 'both'
        """
        if entity_id not in self.graph:
            return set()
        
        neighbors = set()
        current_level = {entity_id}
        
        for _ in range(depth):
            next_level = set()
            for node in current_level:
                if direction in ['out', 'both']:
                    next_level.update(self.graph.successors(node))
                if direction in ['in', 'both']:
                    next_level.update(self.graph.predecessors(node))
            neighbors.update(next_level)
            current_level = next_level - neighbors
        
        neighbors.discard(entity_id)  # Remove the starting entity
        return neighbors
    
    def get_subgraph(self, entity_ids: List[str], include_neighbors: bool = True,
                     neighbor_depth: int = 1) -> nx.DiGraph:
        """Extract a subgraph containing the specified entities."""
        nodes = set(entity_ids)
        
        if include_neighbors:
            for entity_id in entity_ids:
                nodes.update(self.get_neighbors(entity_id, depth=neighbor_depth))
        
        return self.graph.subgraph(nodes).copy()
    
    def find_paths(self, source_id: str, target_id: str, 
                   max_length: int = 3) -> List[List[str]]:
        """Find all paths between two entities up to a maximum length."""
        if source_id not in self.graph or target_id not in self.graph:
            return []
        
        try:
            paths = list(nx.all_simple_paths(
                self.graph, source_id, target_id, cutoff=max_length
            ))
            return paths
        except nx.NetworkXNoPath:
            return []
    
    def search_by_entity(self, query_entities: List[str], 
                         depth: int = 2, k: int = 10) -> List[RetrievalResult]:
        """
        Search the graph starting from query entities.
        
        Returns entities and relationships within the specified depth,
        scored by relevance (closeness to query entities).
        """
        results = []
        visited = set()
        
        # Find matching entities
        matched_entity_ids = []
        for query in query_entities:
            entity = self.get_entity_by_name(query)
            if entity:
                matched_entity_ids.append(entity.id)
        
        if not matched_entity_ids:
            return results
        
        # BFS to find related entities with distance-based scoring
        for start_id in matched_entity_ids:
            queue = [(start_id, 0)]  # (entity_id, distance)
            
            while queue and len(results) < k * 2:
                current_id, dist = queue.pop(0)
                
                if current_id in visited or dist > depth:
                    continue
                visited.add(current_id)
                
                if current_id in self.entities:
                    entity = self.entities[current_id]
                    # Score decreases with distance
                    score = 1.0 / (1.0 + dist)
                    
                    results.append(RetrievalResult(
                        id=entity.id,
                        content=entity.to_text(),
                        score=score,
                        source='graph',
                        metadata={
                            'name': entity.name,
                            'type': entity.type,
                            'distance': dist
                        }
                    ))
                
                # Add neighbors to queue
                for neighbor in self.get_neighbors(current_id, depth=1):
                    if neighbor not in visited:
                        queue.append((neighbor, dist + 1))
        
        # Sort by score and return top k
        results.sort(key=lambda x: x.score, reverse=True)
        return results[:k]
    
    def get_relationship_context(self, entity_ids: List[str]) -> List[RetrievalResult]:
        """Get relationships involving the specified entities."""
        results = []
        seen_rels = set()
        
        for entity_id in entity_ids:
            # Outgoing relationships
            for _, target, data in self.graph.out_edges(entity_id, data=True):
                rel_id = data.get('id')
                if rel_id and rel_id not in seen_rels:
                    seen_rels.add(rel_id)
                    rel = self.relationships.get(rel_id)
                    if rel:
                        results.append(RetrievalResult(
                            id=rel.id,
                            content=rel.to_text(),
                            score=rel.weight,
                            source='graph',
                            metadata={
                                'source': rel.source,
                                'target': rel.target,
                                'relation_type': rel.relation_type
                            }
                        ))
            
            # Incoming relationships
            for source, _, data in self.graph.in_edges(entity_id, data=True):
                rel_id = data.get('id')
                if rel_id and rel_id not in seen_rels:
                    seen_rels.add(rel_id)
                    rel = self.relationships.get(rel_id)
                    if rel:
                        results.append(RetrievalResult(
                            id=rel.id,
                            content=rel.to_text(),
                            score=rel.weight,
                            source='graph',
                            metadata={
                                'source': rel.source,
                                'target': rel.target,
                                'relation_type': rel.relation_type
                            }
                        ))
        
        return results
    
    def get_stats(self) -> Dict[str, Any]:
        """Get graph statistics."""
        return {
            'num_entities': len(self.entities),
            'num_relationships': len(self.relationships),
            'num_nodes': self.graph.number_of_nodes(),
            'num_edges': self.graph.number_of_edges(),
            'density': nx.density(self.graph) if self.graph.number_of_nodes() > 0 else 0
        }


print("✅ KnowledgeGraph class defined!")


## 🔀 6. Hybrid Retriever

The Hybrid Retriever combines vector search and graph search with multiple fusion strategies:
- **Reciprocal Rank Fusion (RRF)**: Combines rankings from multiple sources
- **Weighted Score Fusion**: Weighted average of normalized scores
- **Graph-Enhanced Vector Search**: Uses graph context to boost vector results


In [ ]:
class HybridRetriever:
    """
    Hybrid retrieval system combining vector search and graph-based search.
    
    Fusion Strategies:
    1. RRF (Reciprocal Rank Fusion): Rank-based fusion
    2. Weighted: Score-based weighted combination
    3. Graph-Enhanced: Use graph to boost/filter vector results
    """
    
    def __init__(self, vector_store: VectorStore, knowledge_graph: KnowledgeGraph):
        self.vector_store = vector_store
        self.knowledge_graph = knowledge_graph
        print("✅ HybridRetriever initialized")
    
    def extract_entities_from_query(self, query: str) -> List[str]:
        """
        Extract potential entity names from a query.
        Simple approach: match against known entity names.
        """
        query_lower = query.lower()
        matched_entities = []
        
        for entity_name in self.knowledge_graph.entity_name_to_id.keys():
            if entity_name in query_lower:
                matched_entities.append(entity_name)
        
        return matched_entities
    
    def vector_search(self, query: str, collection: str = 'documents', 
                      k: int = 5) -> List[RetrievalResult]:
        """Perform vector similarity search."""
        return self.vector_store.search(collection, query, k)
    
    def graph_search(self, query: str, depth: int = 2, 
                     k: int = 5) -> List[RetrievalResult]:
        """Perform graph-based search starting from entities in the query."""
        entities = self.extract_entities_from_query(query)
        
        if not entities:
            # Fall back to searching entities collection by vector similarity
            entity_results = self.vector_store.search('entities', query, k=3)
            entities = [r.metadata.get('name', '') for r in entity_results if r.metadata.get('name')]
        
        results = self.knowledge_graph.search_by_entity(entities, depth=depth, k=k)
        
        # Also add relationship context
        entity_ids = [self.knowledge_graph.entity_name_to_id.get(e.lower()) 
                      for e in entities if e.lower() in self.knowledge_graph.entity_name_to_id]
        entity_ids = [eid for eid in entity_ids if eid]
        
        if entity_ids:
            rel_results = self.knowledge_graph.get_relationship_context(entity_ids)
            results.extend(rel_results)
        
        return results[:k]
    
    def reciprocal_rank_fusion(self, result_lists: List[List[RetrievalResult]], 
                                k: int = 60) -> List[RetrievalResult]:
        """
        Reciprocal Rank Fusion (RRF) algorithm.
        
        Combines multiple ranked lists using: score = sum(1 / (k + rank))
        """
        scores: Dict[str, float] = defaultdict(float)
        result_map: Dict[str, RetrievalResult] = {}
        
        for result_list in result_lists:
            for rank, result in enumerate(result_list):
                scores[result.id] += 1.0 / (k + rank + 1)
                if result.id not in result_map:
                    result_map[result.id] = result
        
        # Create fused results
        fused_results = []
        for result_id, score in sorted(scores.items(), key=lambda x: x[1], reverse=True):
            original = result_map[result_id]
            fused_results.append(RetrievalResult(
                id=result_id,
                content=original.content,
                score=score,
                source='hybrid_rrf',
                metadata={**original.metadata, 'original_source': original.source}
            ))
        
        return fused_results
    
    def weighted_fusion(self, vector_results: List[RetrievalResult],
                        graph_results: List[RetrievalResult],
                        vector_weight: float = 0.6,
                        graph_weight: float = 0.4) -> List[RetrievalResult]:
        """
        Weighted score fusion with normalized scores.
        """
        scores: Dict[str, float] = defaultdict(float)
        result_map: Dict[str, RetrievalResult] = {}
        
        # Normalize and weight vector results
        if vector_results:
            max_v_score = max(r.score for r in vector_results)
            for result in vector_results:
                norm_score = result.score / max_v_score if max_v_score > 0 else 0
                scores[result.id] += vector_weight * norm_score
                result_map[result.id] = result
        
        # Normalize and weight graph results
        if graph_results:
            max_g_score = max(r.score for r in graph_results)
            for result in graph_results:
                norm_score = result.score / max_g_score if max_g_score > 0 else 0
                scores[result.id] += graph_weight * norm_score
                if result.id not in result_map:
                    result_map[result.id] = result
        
        # Create fused results
        fused_results = []
        for result_id, score in sorted(scores.items(), key=lambda x: x[1], reverse=True):
            original = result_map[result_id]
            fused_results.append(RetrievalResult(
                id=result_id,
                content=original.content,
                score=score,
                source='hybrid_weighted',
                metadata={**original.metadata, 'original_source': original.source}
            ))
        
        return fused_results
    
    def graph_enhanced_search(self, query: str, k: int = 5,
                              boost_factor: float = 1.5) -> List[RetrievalResult]:
        """
        Graph-enhanced vector search.
        
        Uses graph context to boost scores of vector results that are
        connected to entities mentioned in the query.
        """
        # Get vector results
        vector_results = self.vector_search(query, k=k*2)
        
        # Extract entities from query
        query_entities = self.extract_entities_from_query(query)
        
        if not query_entities:
            return vector_results[:k]
        
        # Get entity IDs
        query_entity_ids = set()
        for name in query_entities:
            eid = self.knowledge_graph.entity_name_to_id.get(name.lower())
            if eid:
                query_entity_ids.add(eid)
                # Also add neighbors
                query_entity_ids.update(
                    self.knowledge_graph.get_neighbors(eid, depth=1)
                )
        
        # Boost scores for results mentioning connected entities
        boosted_results = []
        for result in vector_results:
            boost = 1.0
            content_lower = result.content.lower()
            
            # Check if result mentions any connected entities
            for entity_id in query_entity_ids:
                entity = self.knowledge_graph.entities.get(entity_id)
                if entity and entity.name.lower() in content_lower:
                    boost = boost_factor
                    break
            
            boosted_results.append(RetrievalResult(
                id=result.id,
                content=result.content,
                score=result.score * boost,
                source='hybrid_graph_enhanced',
                metadata={**result.metadata, 'boost_applied': boost > 1.0}
            ))
        
        boosted_results.sort(key=lambda x: x.score, reverse=True)
        return boosted_results[:k]
    
    def retrieve(self, query: str, k: int = 5,
                 method: str = 'rrf',
                 vector_weight: float = 0.6) -> HybridResult:
        """
        Perform hybrid retrieval with the specified fusion method.
        
        Args:
            query: Search query
            k: Number of results to return
            method: 'rrf', 'weighted', or 'graph_enhanced'
            vector_weight: Weight for vector results (weighted fusion only)
        """
        # Get results from both sources
        vector_results = self.vector_search(query, k=k*2)
        graph_results = self.graph_search(query, k=k*2)
        
        # Apply fusion
        if method == 'rrf':
            fused = self.reciprocal_rank_fusion([vector_results, graph_results])
        elif method == 'weighted':
            fused = self.weighted_fusion(
                vector_results, graph_results,
                vector_weight=vector_weight,
                graph_weight=1-vector_weight
            )
        elif method == 'graph_enhanced':
            fused = self.graph_enhanced_search(query, k=k)
        else:
            raise ValueError(f"Unknown fusion method: {method}")
        
        return HybridResult(
            query=query,
            vector_results=vector_results[:k],
            graph_results=graph_results[:k],
            fused_results=fused[:k],
            fusion_method=method
        )


print("✅ HybridRetriever class defined!")


## 📊 7. Entity Extraction & Knowledge Graph Building

For this demo, we define entities and relationships manually (simulating what GraphRAG would extract with an LLM). In production, you would use an LLM to extract these from your documents.


In [ ]:
# Define entities (extracted from documents)
ENTITIES = [
    # People
    Entity("e1", "Sarah Chen", "PERSON", 
           "CEO of TechCorp, co-founder, previously worked at Google and Stanford AI Lab",
           source_docs=["doc_1", "doc_2", "doc_6"]),
    Entity("e2", "Michael Rodriguez", "PERSON",
           "CTO of TechCorp, co-founder, PhD in Machine Learning from MIT, oversees R&D",
           source_docs=["doc_1", "doc_2", "doc_7"]),
    Entity("e3", "Jennifer Park", "PERSON",
           "CFO of TechCorp, joined 2019 from Goldman Sachs, led successful IPO",
           source_docs=["doc_2", "doc_5"]),
    Entity("e4", "David Thompson", "PERSON",
           "Head of Sales at TechCorp, expanded customer base to Fortune 500",
           source_docs=["doc_2", "doc_8"]),
    Entity("e5", "Dr. Emily Watson", "PERSON",
           "Head of R&D at TechCorp, published 50+ papers, breakthrough in transformer architectures",
           source_docs=["doc_4"]),
    
    # Organizations
    Entity("e6", "TechCorp", "ORGANIZATION",
           "Leading AI technology company founded in 2015, 5000+ employees, $2.5B revenue",
           source_docs=["doc_1", "doc_2", "doc_3", "doc_4", "doc_5", "doc_6", "doc_7", "doc_8"]),
    Entity("e7", "Google", "ORGANIZATION",
           "Tech company where Sarah Chen previously worked",
           source_docs=["doc_2"]),
    Entity("e8", "Stanford AI Lab", "ORGANIZATION",
           "Research institution where Sarah Chen previously worked",
           source_docs=["doc_2"]),
    Entity("e9", "MIT", "ORGANIZATION",
           "University where Michael Rodriguez got his PhD, research partner",
           source_docs=["doc_2", "doc_4"]),
    Entity("e10", "Goldman Sachs", "ORGANIZATION",
           "Investment bank where Jennifer Park worked before joining TechCorp",
           source_docs=["doc_2"]),
    Entity("e11", "Stanford University", "ORGANIZATION",
           "Research partner of TechCorp",
           source_docs=["doc_4"]),
    Entity("e12", "Carnegie Mellon", "ORGANIZATION",
           "Research partner of TechCorp",
           source_docs=["doc_4"]),
    Entity("e13", "Sequoia Capital", "ORGANIZATION",
           "Major investor in TechCorp",
           source_docs=["doc_5"]),
    Entity("e14", "Andreessen Horowitz", "ORGANIZATION",
           "Major investor in TechCorp",
           source_docs=["doc_5"]),
    Entity("e15", "SoftBank Vision Fund", "ORGANIZATION",
           "Major investor in TechCorp",
           source_docs=["doc_5"]),
    
    # Products
    Entity("e16", "AIAssist Pro", "PRODUCT",
           "TechCorp's flagship enterprise AI assistant for customer service automation, 200+ customers",
           source_docs=["doc_3", "doc_7"]),
    Entity("e17", "DataSense Analytics", "PRODUCT",
           "TechCorp's predictive analytics product for supply chain optimization",
           source_docs=["doc_3"]),
    Entity("e18", "SecureAI", "PRODUCT",
           "TechCorp's AI-powered cybersecurity product launched in 2024",
           source_docs=["doc_3"]),
    
    # Clients
    Entity("e19", "Walmart", "ORGANIZATION",
           "Major retail client of TechCorp using DataSense Analytics",
           source_docs=["doc_3", "doc_8"]),
    Entity("e20", "Target", "ORGANIZATION",
           "Retail client using DataSense Analytics, reported 30% efficiency improvements",
           source_docs=["doc_3", "doc_8"]),
    Entity("e21", "JPMorgan Chase", "ORGANIZATION",
           "Banking client using SecureAI",
           source_docs=["doc_3", "doc_8"]),
    Entity("e22", "Bank of America", "ORGANIZATION",
           "Banking client using SecureAI",
           source_docs=["doc_3", "doc_8"]),
    Entity("e23", "Wells Fargo", "ORGANIZATION",
           "Banking client using SecureAI",
           source_docs=["doc_3", "doc_8"]),
    Entity("e24", "FedEx", "ORGANIZATION",
           "Logistics partner for autonomous systems pilot",
           source_docs=["doc_6", "doc_8"]),
    Entity("e25", "UPS", "ORGANIZATION",
           "Logistics partner for autonomous systems pilot",
           source_docs=["doc_6", "doc_8"]),
    Entity("e26", "Mayo Clinic", "ORGANIZATION",
           "Healthcare partner for AI expansion",
           source_docs=["doc_6"]),
    Entity("e27", "Cleveland Clinic", "ORGANIZATION",
           "Healthcare partner for AI expansion",
           source_docs=["doc_6"]),
    Entity("e28", "Amazon", "ORGANIZATION",
           "Strategic partner and client",
           source_docs=["doc_8"]),
    Entity("e29", "Microsoft", "ORGANIZATION",
           "Strategic partner",
           source_docs=["doc_8"]),
    
    # Locations
    Entity("e30", "San Francisco", "LOCATION",
           "TechCorp headquarters location",
           source_docs=["doc_1"]),
    Entity("e31", "London", "LOCATION",
           "Planned R&D center location",
           source_docs=["doc_6"]),
    Entity("e32", "Singapore", "LOCATION",
           "Planned R&D center location",
           source_docs=["doc_6"]),
    Entity("e33", "Tel Aviv", "LOCATION",
           "Planned R&D center location",
           source_docs=["doc_6"]),
]

print(f"📊 Defined {len(ENTITIES)} entities")
for entity_type in set(e.type for e in ENTITIES):
    count = sum(1 for e in ENTITIES if e.type == entity_type)
    print(f"   - {entity_type}: {count}")


In [ ]:
# Define relationships
RELATIONSHIPS = [
    # Leadership relationships
    Relationship("r1", "e1", "e6", "CEO_OF", "Sarah Chen is the CEO of TechCorp"),
    Relationship("r2", "e2", "e6", "CTO_OF", "Michael Rodriguez is the CTO of TechCorp"),
    Relationship("r3", "e3", "e6", "CFO_OF", "Jennifer Park is the CFO of TechCorp"),
    Relationship("r4", "e4", "e6", "LEADS_SALES", "David Thompson leads Sales at TechCorp"),
    Relationship("r5", "e5", "e6", "LEADS_RD", "Dr. Emily Watson leads R&D at TechCorp"),
    
    # Founding relationships
    Relationship("r6", "e1", "e6", "FOUNDED", "Sarah Chen co-founded TechCorp in 2015"),
    Relationship("r7", "e2", "e6", "FOUNDED", "Michael Rodriguez co-founded TechCorp in 2015"),
    
    # Previous employment
    Relationship("r8", "e1", "e7", "WORKED_AT", "Sarah Chen previously worked at Google"),
    Relationship("r9", "e1", "e8", "WORKED_AT", "Sarah Chen previously worked at Stanford AI Lab"),
    Relationship("r10", "e2", "e9", "EDUCATED_AT", "Michael Rodriguez got PhD from MIT"),
    Relationship("r11", "e3", "e10", "WORKED_AT", "Jennifer Park worked at Goldman Sachs before TechCorp"),
    
    # Product relationships
    Relationship("r12", "e6", "e16", "DEVELOPS", "TechCorp develops AIAssist Pro"),
    Relationship("r13", "e6", "e17", "DEVELOPS", "TechCorp develops DataSense Analytics"),
    Relationship("r14", "e6", "e18", "DEVELOPS", "TechCorp develops SecureAI"),
    Relationship("r15", "e2", "e16", "ARCHITECTED", "Michael Rodriguez architected AIAssist Pro platform"),
    
    # Client relationships
    Relationship("r16", "e19", "e17", "USES", "Walmart uses DataSense Analytics"),
    Relationship("r17", "e20", "e17", "USES", "Target uses DataSense Analytics"),
    Relationship("r18", "e21", "e18", "USES", "JPMorgan Chase uses SecureAI"),
    Relationship("r19", "e22", "e18", "USES", "Bank of America uses SecureAI"),
    Relationship("r20", "e23", "e18", "USES", "Wells Fargo uses SecureAI"),
    
    # Partnership relationships
    Relationship("r21", "e6", "e11", "PARTNERS_WITH", "TechCorp collaborates with Stanford University"),
    Relationship("r22", "e6", "e9", "PARTNERS_WITH", "TechCorp collaborates with MIT"),
    Relationship("r23", "e6", "e12", "PARTNERS_WITH", "TechCorp collaborates with Carnegie Mellon"),
    Relationship("r24", "e6", "e24", "PARTNERS_WITH", "TechCorp partners with FedEx for logistics AI"),
    Relationship("r25", "e6", "e25", "PARTNERS_WITH", "TechCorp partners with UPS for logistics AI"),
    Relationship("r26", "e6", "e26", "PARTNERS_WITH", "TechCorp partners with Mayo Clinic for healthcare AI"),
    Relationship("r27", "e6", "e27", "PARTNERS_WITH", "TechCorp partners with Cleveland Clinic"),
    Relationship("r28", "e6", "e28", "PARTNERS_WITH", "TechCorp has strategic partnership with Amazon"),
    Relationship("r29", "e6", "e29", "PARTNERS_WITH", "TechCorp has strategic partnership with Microsoft"),
    
    # Investment relationships
    Relationship("r30", "e13", "e6", "INVESTED_IN", "Sequoia Capital invested in TechCorp"),
    Relationship("r31", "e14", "e6", "INVESTED_IN", "Andreessen Horowitz invested in TechCorp"),
    Relationship("r32", "e15", "e6", "INVESTED_IN", "SoftBank Vision Fund invested in TechCorp"),
    
    # Location relationships
    Relationship("r33", "e6", "e30", "HEADQUARTERED_IN", "TechCorp is headquartered in San Francisco"),
    Relationship("r34", "e6", "e31", "EXPANDING_TO", "TechCorp plans R&D center in London"),
    Relationship("r35", "e6", "e32", "EXPANDING_TO", "TechCorp plans R&D center in Singapore"),
    Relationship("r36", "e6", "e33", "EXPANDING_TO", "TechCorp plans R&D center in Tel Aviv"),
]

print(f"🔗 Defined {len(RELATIONSHIPS)} relationships")
for rel_type in set(r.relation_type for r in RELATIONSHIPS):
    count = sum(1 for r in RELATIONSHIPS if r.relation_type == rel_type)
    print(f"   - {rel_type}: {count}")


## 🏗️ 8. Building the Hybrid Retrieval System

Now let's initialize all components and populate them with our data.


In [ ]:
# Initialize the Vector Store
print("=" * 60)
print("Initializing Vector Store...")
print("=" * 60)
vector_store = VectorStore(embedding_model="all-MiniLM-L6-v2")

# Add documents to vector store
print("\n📄 Adding documents to vector store...")
vector_store.add_documents('documents', [
    {'id': doc['id'], 'content': doc['content'], 'metadata': {'title': doc['title']}}
    for doc in DOCUMENTS
])

# Add entities to vector store
print("\n🏷️ Adding entities to vector store...")
vector_store.add_entities(ENTITIES)

# Add relationships to vector store
print("\n🔗 Adding relationships to vector store...")
vector_store.add_relationships(RELATIONSHIPS)

print("\n✅ Vector Store populated!")


In [ ]:
# Initialize the Knowledge Graph
print("=" * 60)
print("Initializing Knowledge Graph...")
print("=" * 60)
knowledge_graph = KnowledgeGraph()

# Add entities
print("\n🏷️ Adding entities to graph...")
for entity in ENTITIES:
    knowledge_graph.add_entity(entity)

# Add relationships
print("🔗 Adding relationships to graph...")
for relationship in RELATIONSHIPS:
    knowledge_graph.add_relationship(relationship)

# Print graph statistics
stats = knowledge_graph.get_stats()
print(f"\n📊 Graph Statistics:")
print(f"   Entities: {stats['num_entities']}")
print(f"   Relationships: {stats['num_relationships']}")
print(f"   Graph Density: {stats['density']:.4f}")

print("\n✅ Knowledge Graph populated!")


In [ ]:
# Initialize the Hybrid Retriever
print("=" * 60)
print("Initializing Hybrid Retriever...")
print("=" * 60)
hybrid_retriever = HybridRetriever(vector_store, knowledge_graph)

print("\n🎉 Hybrid Retrieval System Ready!")


## 🔍 9. Demonstration: Hybrid Retrieval in Action

Let's test our hybrid retrieval system with different queries and fusion strategies.


In [ ]:
def display_results(result: HybridResult, show_all: bool = False):
    """Pretty print hybrid retrieval results."""
    print("=" * 70)
    print(f"🔍 QUERY: {result.query}")
    print(f"📊 Fusion Method: {result.fusion_method.upper()}")
    print("=" * 70)
    
    if show_all:
        print("\n📌 VECTOR SEARCH RESULTS:")
        print("-" * 50)
        for i, r in enumerate(result.vector_results[:5], 1):
            print(f"  {i}. [{r.score:.3f}] {r.content[:80]}...")
        
        print("\n📌 GRAPH SEARCH RESULTS:")
        print("-" * 50)
        for i, r in enumerate(result.graph_results[:5], 1):
            print(f"  {i}. [{r.score:.3f}] {r.content[:80]}...")
    
    print("\n🎯 FUSED RESULTS (Top 5):")
    print("-" * 50)
    for i, r in enumerate(result.fused_results[:5], 1):
        orig_source = r.metadata.get('original_source', 'unknown')
        print(f"  {i}. [{r.score:.3f}] (from: {orig_source})")
        print(f"      {r.content[:100]}...")
        print()


# Test Query 1: Entity-focused query
query1 = "Who is Sarah Chen and what is her role?"
print("\n" + "🔸" * 35)
print("TEST 1: Entity-focused query with RRF fusion")
print("🔸" * 35)
result1 = hybrid_retriever.retrieve(query1, k=5, method='rrf')
display_results(result1, show_all=True)


In [ ]:
# Test Query 2: Product/relationship query with Weighted fusion
query2 = "What products does TechCorp offer and who are the clients?"
print("\n" + "🔸" * 35)
print("TEST 2: Product query with WEIGHTED fusion (70% vector, 30% graph)")
print("🔸" * 35)
result2 = hybrid_retriever.retrieve(query2, k=5, method='weighted', vector_weight=0.7)
display_results(result2, show_all=True)


In [ ]:
# Test Query 3: Graph-enhanced search (best for relationship queries)
query3 = "What are TechCorp's partnerships with universities?"
print("\n" + "🔸" * 35)
print("TEST 3: Relationship query with GRAPH-ENHANCED search")
print("🔸" * 35)
result3 = hybrid_retriever.retrieve(query3, k=5, method='graph_enhanced')
display_results(result3, show_all=True)


In [ ]:
# Test Query 4: Compare all fusion methods on the same query
query4 = "Who founded TechCorp and what are the company's main investors?"
print("\n" + "🔹" * 35)
print("TEST 4: Comparing ALL fusion methods on the same query")
print("🔹" * 35)

methods = ['rrf', 'weighted', 'graph_enhanced']
for method in methods:
    result = hybrid_retriever.retrieve(query4, k=3, method=method)
    print(f"\n📊 {method.upper()} Results:")
    print("-" * 40)
    for i, r in enumerate(result.fused_results[:3], 1):
        print(f"  {i}. [{r.score:.3f}] {r.content[:70]}...")


## 🎨 10. Knowledge Graph Visualization

Let's visualize the knowledge graph to understand the entity relationships.


In [ ]:
def visualize_knowledge_graph(kg: KnowledgeGraph, figsize: Tuple[int, int] = (16, 12)):
    """Visualize the knowledge graph with colored nodes by entity type."""
    G = kg.graph
    
    # Define colors for entity types
    type_colors = {
        'PERSON': '#FF6B6B',       # Red
        'ORGANIZATION': '#4ECDC4', # Teal
        'PRODUCT': '#45B7D1',      # Blue
        'LOCATION': '#96CEB4',     # Green
    }
    
    # Get node colors
    node_colors = []
    for node in G.nodes():
        entity = kg.entities.get(node)
        if entity:
            node_colors.append(type_colors.get(entity.type, '#CCCCCC'))
        else:
            node_colors.append('#CCCCCC')
    
    # Get node labels (use names instead of IDs)
    labels = {}
    for node in G.nodes():
        entity = kg.entities.get(node)
        if entity:
            # Truncate long names
            name = entity.name
            if len(name) > 15:
                name = name[:12] + "..."
            labels[node] = name
        else:
            labels[node] = node
    
    # Create figure
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    
    # Layout
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Draw edges with arrows
    nx.draw_networkx_edges(G, pos, ax=ax, 
                           edge_color='#CCCCCC', 
                           arrows=True,
                           arrowsize=15,
                           connectionstyle="arc3,rad=0.1",
                           alpha=0.6)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, ax=ax,
                           node_color=node_colors,
                           node_size=800,
                           alpha=0.9)
    
    # Draw labels
    nx.draw_networkx_labels(G, pos, labels, ax=ax,
                           font_size=8,
                           font_weight='bold')
    
    # Add legend
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w',
                                  markerfacecolor=color, markersize=12, label=etype)
                      for etype, color in type_colors.items()]
    ax.legend(handles=legend_elements, loc='upper left', title='Entity Types')
    
    ax.set_title('TechCorp Knowledge Graph', fontsize=16, fontweight='bold')
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return fig


# Visualize the knowledge graph
fig = visualize_knowledge_graph(knowledge_graph)


In [ ]:
# Visualize a focused subgraph around TechCorp
def visualize_subgraph(kg: KnowledgeGraph, center_entity: str, depth: int = 1):
    """Visualize a subgraph centered on a specific entity."""
    entity = kg.get_entity_by_name(center_entity)
    if not entity:
        print(f"Entity '{center_entity}' not found")
        return
    
    # Get subgraph
    subgraph = kg.get_subgraph([entity.id], include_neighbors=True, neighbor_depth=depth)
    
    # Define colors
    type_colors = {
        'PERSON': '#FF6B6B',
        'ORGANIZATION': '#4ECDC4',
        'PRODUCT': '#45B7D1',
        'LOCATION': '#96CEB4',
    }
    
    # Setup
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Node colors and sizes
    node_colors = []
    node_sizes = []
    labels = {}
    
    for node in subgraph.nodes():
        e = kg.entities.get(node)
        if e:
            node_colors.append(type_colors.get(e.type, '#CCCCCC'))
            # Make center entity larger
            node_sizes.append(1500 if node == entity.id else 800)
            labels[node] = e.name if len(e.name) <= 15 else e.name[:12] + "..."
    
    pos = nx.spring_layout(subgraph, k=2, seed=42)
    
    # Draw
    nx.draw_networkx_edges(subgraph, pos, ax=ax, edge_color='#888888', 
                           arrows=True, arrowsize=20, alpha=0.7)
    nx.draw_networkx_nodes(subgraph, pos, ax=ax, node_color=node_colors,
                           node_size=node_sizes, alpha=0.9)
    nx.draw_networkx_labels(subgraph, pos, labels, ax=ax, font_size=9, font_weight='bold')
    
    # Edge labels
    edge_labels = {}
    for u, v, data in subgraph.edges(data=True):
        rel_type = data.get('relation_type', '')
        edge_labels[(u, v)] = rel_type
    nx.draw_networkx_edge_labels(subgraph, pos, edge_labels, ax=ax, font_size=7)
    
    ax.set_title(f"Subgraph: {center_entity} (depth={depth})", fontsize=14, fontweight='bold')
    ax.axis('off')
    plt.tight_layout()
    plt.show()


# Show TechCorp's immediate relationships
print("📊 TechCorp's 1-hop neighborhood:")
visualize_subgraph(knowledge_graph, "TechCorp", depth=1)


## 📊 11. Performance Analysis

Let's analyze how the different retrieval methods perform.


In [ ]:
import time

def benchmark_retrieval(retriever: HybridRetriever, queries: List[str], methods: List[str], runs: int = 5):
    """Benchmark retrieval methods."""
    results = {method: {'times': [], 'avg_score': []} for method in methods}
    
    for query in queries:
        for method in methods:
            times = []
            scores = []
            for _ in range(runs):
                start = time.time()
                result = retriever.retrieve(query, k=5, method=method)
                elapsed = time.time() - start
                times.append(elapsed)
                if result.fused_results:
                    scores.append(np.mean([r.score for r in result.fused_results]))
            
            results[method]['times'].extend(times)
            results[method]['avg_score'].extend(scores)
    
    return results


# Benchmark queries
benchmark_queries = [
    "Who is the CEO of TechCorp?",
    "What products does TechCorp develop?",
    "Which banks use TechCorp's cybersecurity product?",
    "Who are TechCorp's investors?",
    "What universities does TechCorp partner with?"
]

print("⏱️ Running benchmark (this may take a few seconds)...")
benchmark_results = benchmark_retrieval(hybrid_retriever, benchmark_queries, 
                                        ['rrf', 'weighted', 'graph_enhanced'], runs=3)

# Display results
print("\n" + "=" * 60)
print("📊 BENCHMARK RESULTS")
print("=" * 60)
print(f"\n{'Method':<20} {'Avg Time (ms)':<15} {'Avg Score':<15}")
print("-" * 50)
for method, data in benchmark_results.items():
    avg_time = np.mean(data['times']) * 1000  # Convert to ms
    avg_score = np.mean(data['avg_score']) if data['avg_score'] else 0
    print(f"{method:<20} {avg_time:<15.2f} {avg_score:<15.4f}")


## 📚 12. Summary & Best Practices

### Key Concepts

1. **Vector Search (ChromaDB)**
   - Uses sentence embeddings for semantic similarity
   - Best for: Finding semantically similar content, paraphrase matching
   - Limitation: Doesn't capture explicit relationships

2. **Graph Search (NetworkX)**
   - Uses graph traversal for relationship-based retrieval
   - Best for: Multi-hop reasoning, relationship queries
   - Limitation: Requires entity extraction, less semantic flexibility

3. **Hybrid Retrieval**
   - Combines both approaches for comprehensive retrieval
   - Multiple fusion strategies for different use cases

### Fusion Strategy Guide

| Strategy | Best For | Trade-off |
|----------|----------|-----------|
| **RRF** | General queries, balanced results | Equal weight to all sources |
| **Weighted** | Tuning semantic vs. structural preference | Requires weight tuning |
| **Graph-Enhanced** | Relationship-heavy queries | Depends on entity extraction quality |

### Architecture Diagram

```
┌─────────────────────────────────────────────────────────────┐
│                    HYBRID RETRIEVAL SYSTEM                  │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  ┌──────────────┐        ┌──────────────┐                   │
│  │   ChromaDB   │        │   NetworkX   │                   │
│  │ Vector Store │        │ Knowledge    │                   │
│  │              │        │ Graph        │                   │
│  │ • Documents  │        │              │                   │
│  │ • Entities   │        │ • Entities   │                   │
│  │ • Relations  │        │ • Relations  │                   │
│  └──────┬───────┘        └──────┬───────┘                   │
│         │                       │                           │
│         │ Semantic Search       │ Graph Traversal           │
│         │                       │                           │
│         └───────────┬───────────┘                           │
│                     │                                       │
│              ┌──────▼──────┐                                │
│              │   FUSION    │                                │
│              │  • RRF      │                                │
│              │  • Weighted │                                │
│              │  • Enhanced │                                │
│              └──────┬──────┘                                │
│                     │                                       │
│              ┌──────▼──────┐                                │
│              │   RESULTS   │                                │
│              └─────────────┘                                │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Production Considerations

1. **Entity Extraction**: Use LLMs (like GPT-4) for automatic entity extraction
2. **Scaling**: Replace in-memory stores with persistent solutions (Chroma persistent, Neo4j)
3. **Caching**: Cache embeddings and frequently accessed graph paths
4. **Monitoring**: Track retrieval quality metrics (MRR, recall@k)


In [ ]:
print("🎉 Notebook Complete!")
print("\n📚 Key Takeaways:")
print("   1. Vector search excels at semantic similarity (finding related content)")
print("   2. Graph search excels at relationship traversal (finding connected entities)")  
print("   3. Hybrid retrieval combines both for comprehensive results")
print("   4. Different fusion strategies suit different query types")
print("   5. RRF is a good default; Weighted allows fine-tuning; Graph-Enhanced boosts relationships")
print("\n🔗 Resources:")
print("   - ChromaDB: https://docs.trychroma.com/")
print("   - NetworkX: https://networkx.org/")
print("   - Sentence Transformers: https://www.sbert.net/")
print("   - Microsoft GraphRAG: https://github.com/microsoft/graphrag")
